### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드
콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다


In [19]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

# 0단계: 문서 로드
pdf_path = Path("data") / "콘텐츠분쟁해결_사례.pdf"
documents = loader.load()

# 1단계: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = text_splitter.split_documents(documents)

# 2단계: 임베딩 모델
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)

# 3단계: 벡터스토어 생성
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4단계: 검색기
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# 5단계: LLM 설정
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

# 6단계: 프롬프트 템플릿
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# 7단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 8단계: 분쟁 유형 분류 함수
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# 9단계: 테스트 질문
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

# 10단계: 실행
for i, question in enumerate(test_questions, 1):
    dispute_type = classify_dispute_type(question)
    print(f"\n【테스트 {i}/5】")
    print(f"▶ 질문: {question}")
    print(f"▶ 분쟁 유형: {dispute_type}")
    
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]

    print(f"\n🧑‍⚖️ 답변:")
    print("-" * 50)
    print(answer)

    print(f"\n📄 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get("page", "N/A")
        preview = doc.page_content[:100].replace("\n", " ")
        print(f"  {j}. 페이지 {page}: {preview}...")

    print("-" * 50)



【테스트 1/5】
▶ 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
▶ 분쟁 유형: 게임

🧑‍⚖️ 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 근거로 답변하겠습니다.

### 1. 사례 분석
- **사례 1 (2006년)**: 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 거부된 사례입니다. 계정 명의자가 직접 복구 요청을 해야 한다는 점이 강조되었습니다.
- **사례 2 (2009년)**: 시스템 오류로 게임머니가 소실되었다고 주장했으나, 시스템 오류가 발생하지 않았다는 게임사의 입장이었습니다. 사용 내역을 통해 오류가 없음을 확인하였습니다.
- **사례 3 (2009년)**: 해킹으로 인한 아이템 손실이 발생했으나, 보안서비스 미사용으로 복구가 거부된 사례입니다.
- **사례 4 (2007년)**: 해킹 아이템을 구매한 후 회수된 사례로, 게임머니가 환급되어 피해가 없다고 판단되었습니다.
- **사례 5 (2008년)**: 보안서비스 미작동으로 아이템 도난이 발생한 경우, 게임사가 아이템을 복구해 준 사례입니다.

### 2. 관련 법령 및 조항
- **민법 제250조 및 제251조**: 도품이나 유실물에 대한 반환 청구에 관한 규정이 있으나, 이는 주로 물리적 재화에 적용됩니다.
- **계약법 및 소비자 보호법**: 게임사의 약관과 소비자 보호 관련 법령이 적용될 수 있습니다.

### 3. 실무적 해결방안
1. **계정 명의 확인**: 계정 명의자가 본인인지 확인하고, 명의자가 아닌 경우 명의자에게 복구 요청을 하도록 합니다.
2. **증거 수집**: 시스템 오류가 발생한 시점의 스크린샷, 로그 기록 등을 수집하여 오류 발생을 입증할 수 있는 증거를 확보합니다.